In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df = spark.read.format("parquet").load("s3://argha-associatede-s3/bronze/products/")

display(df)

In [0]:
df.createOrReplaceTempView("products")


### Functions

In [0]:
%sql

CREATE OR REPLACE FUNCTION aws_db_retail_metastore.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql

select product_id, aws_db_retail_metastore.bronze.discount_func(price) as discounted_price
from products

In [0]:
# using sql function in pyspark

df1 = df.withColumn("discounted_price",expr("aws_db_retail_metastore.bronze.discount_func(price)").alias("discounted_price"))

df1.display()

In [0]:
%sql
-- creating the function to be able to be used by sql directly



CREATE OR REPLACE FUNCTION aws_db_retail_metastore.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
 return p_brand.upper()
$$

In [0]:
df1 = df1.withColumn("brand",expr("aws_db_retail_metastore.bronze.upper_func(brand)"))

In [0]:
df1.display()


### Data Writing

In [0]:
df1.write.format("delta").mode("overwrite").save("s3://argha-associatede-s3/silver/products/") 